In [33]:
import mysql.connector
conn = mysql.connector.connect(host="localhost", user="root", password="")
cursor = conn.cursor()
my_db = "HAKIZIMANA_Emmanuel_lab"
try:
    cursor.execute(f"CREATE DATABASE {my_db} ")
    print("Database created successfully!")
except:
    print(f"Failed to create database called: {my_db}")

Database created successfully!


In [34]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root", 
    password=""
)

cursor = conn.cursor()


cursor.execute("CREATE DATABASE IF NOT EXISTS HAKIZIMANA_Emmanuel_lab")


cursor.execute("USE HAKIZIMANA_Emmanuel_lab")


tables = {
    "User_role": """
    CREATE TABLE IF NOT EXISTS User_role (
        role_id INT PRIMARY KEY, 
        role_name ENUM('admin', 'student', 'technician')
    )
    """,
    
    "users": """
    CREATE TABLE IF NOT EXISTS users (
        userId INT AUTO_INCREMENT PRIMARY KEY,
        fname VARCHAR(255) NOT NULL,
        lname VARCHAR(255) NOT NULL,
        username VARCHAR(255) NOT NULL UNIQUE,
        email VARCHAR(255) NOT NULL UNIQUE,
        phone VARCHAR(255) NOT NULL,
        password VARCHAR(255) NOT NULL,
        role_id INT,
        status VARCHAR(255) NOT NULL DEFAULT 'Active',
        FOREIGN KEY (role_id) REFERENCES User_role(role_id)
            ON UPDATE CASCADE
            ON DELETE SET NULL
    )
    """,
    
    "Computers": """
    CREATE TABLE IF NOT EXISTS Computers(
        computer_id VARCHAR(255) NOT NULL, 
        status ENUM('available', 'reserved', 'under maintenance'),
        PRIMARY KEY (computer_id)
    )
    """,
    
    "Reservation": """
    CREATE TABLE IF NOT EXISTS Reservation (
        reservation_id INT AUTO_INCREMENT PRIMARY KEY,
        user_id INT, 
        computer_id VARCHAR(255), 
        reservation_date DATETIME, 
        reservation_status ENUM('active', 'completed', 'cancelled'),
        FOREIGN KEY (user_id) REFERENCES users(userId),
        FOREIGN KEY (computer_id) REFERENCES Computers(computer_id)
    )
    """,
    
    "Maintenance": """
    CREATE TABLE IF NOT EXISTS Maintenance (
        maintenance_id INT AUTO_INCREMENT PRIMARY KEY,
        computer_id VARCHAR(255), 
        maintenance_date DATETIME, 
        description TEXT,
        FOREIGN KEY (computer_id) REFERENCES Computers(computer_id)
    )
    """
}

try:
    
    for table_name, create_table_sql in tables.items():
        cursor.execute(create_table_sql)

    conn.commit()

    print("All tables created successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:

    cursor.close()
    conn.close()


All tables created successfully!


In [35]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root", 
    password=""
)

cursor = conn.cursor()

cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

cursor.execute("SHOW TABLES")

tables = cursor.fetchall()

if tables:
    print("Tables in the database:")
    for table in tables:
        print(table[0])
else:
    print("No tables found in the database.")

cursor.close()
conn.close()


Tables in the database:
computers
maintenance
reservation
user_role
users


In [ ]:
import mysql.connector
import bcrypt

def hash_password(password):
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), salt)
    return hashed_password

def insert_role_if_not_exists(cursor, role_id, role_name):

    cursor.execute("SELECT COUNT(*) FROM user_role WHERE role_id = %s", (role_id,))
    result = cursor.fetchone()
    
    # If the role doesn't exist, insert it
    if result[0] == 0:
        cursor.execute("INSERT INTO user_role (role_id, role_name) VALUES (%s, %s)", (role_id, role_name))

def insert_user_with_hashed_password(userId, fname, lname, username, email, phone, password, role_id, status):
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root", 
            password=""
        )

        cursor = conn.cursor()

        # Use the database
        cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

   
        insert_role_if_not_exists(cursor, role_id, 'admin' if role_id == 1 else 'student')

        # Hash the password
        hashed_password = hash_password(password)

        # Insert the user into the users table
        cursor.execute("""
            INSERT INTO users (userId, fname, lname, username, email, phone, password, role_id, status)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (userId, fname, lname, username, email, phone, hashed_password, role_id, status))

        conn.commit()

        print("User with hashed password inserted successfully!")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        # Ensure the cursor and connection are closed
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Example usage
insert_user_with_hashed_password(
    userId=1, 
    fname="Emmanuel", 
    lname="Hakizimana", 
    username="emmanuel", 
    email="emmanuel@example.com", 
    phone="1234567890", 
    password="password123", 
    role_id=1, 
    status="Active"
)


User with hashed password inserted successfully!


In [39]:
import mysql.connector

def display_users():
    try:
        # Connect to the MySQL database
        conn = mysql.connector.connect(
            host="localhost",
            user="root", 
            password=""
        )

        cursor = conn.cursor()

        # Use the appropriate database
        cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

        # Execute the query to fetch users and their role names
        cursor.execute("""
            SELECT 
                users.userId, 
                users.fname, 
                users.lname, 
                users.username, 
                users.email, 
                users.phone, 
                users.status, 
                user_role.role_name
            FROM users
            INNER JOIN user_role 
                ON users.role_id = user_role.role_id;
        """)

        # Fetch all results
        results = cursor.fetchall()

        # Display the results
        for row in results:
            print(f"User ID: {row[0]}, Name: {row[1]} {row[2]}, Username: {row[3]}, Email: {row[4]}, Phone: {row[5]}, Status: {row[6]}, Role: {row[7]}")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        # Ensure the cursor and connection are closed
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Call the function to display users
display_users()


User ID: 1, Name: Emmanuel Hakizimana, Username: emmanuel, Email: emmanuel@example.com, Phone: 1234567890, Status: Active, Role: admin


In [40]:
import mysql.connector

try:
    # Connect to the MySQL database
    conn = mysql.connector.connect(
        host="localhost",
        user="root", 
        password=""
    )

    cursor = conn.cursor()

    # Use the appropriate database
    cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

    # Insert data into the Computers table
    cursor.execute("""
        INSERT INTO Computers (computer_id, status)
        VALUES
        ('C001', 'available'),
        ('C002', 'reserved'),
        ('C003', 'under maintenance')
    """)

    # Commit the transaction to save changes to the database
    conn.commit()

    print("Data inserted into Computers table successfully!")

    # Now retrieve and display all data from the Computers table
    cursor.execute("SELECT * FROM Computers")

    # Fetch all rows from the result
    results = cursor.fetchall()

    print("\nAll data from the Computers table:")
    for row in results:
        print(f"Computer ID: {row[0]}, Status: {row[1]}")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Ensure the cursor and connection are closed
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Data inserted into Computers table successfully!

All data from the Computers table:
Computer ID: C001, Status: available
Computer ID: C002, Status: reserved
Computer ID: C003, Status: under maintenance


In [41]:
import mysql.connector

try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root", 
        password=""
    )

    cursor = conn.cursor()
    cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

    cursor.execute("""
        INSERT INTO Maintenance (computer_id, maintenance_date, description)
        VALUES
        ('C003', '2025-03-19 10:00:00', 'Hardware issue, screen replacement')
    """)

    conn.commit()
    print("Data inserted into Maintenance table successfully!")

    cursor.execute("SELECT * FROM Maintenance")
    results = cursor.fetchall()

    print("\nAll data from the Maintenance table:")
    for row in results:
        print(f"Maintenance ID: {row[0]}, Computer ID: {row[1]}, Maintenance Date: {row[2]}, Description: {row[3]}")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Data inserted into Maintenance table successfully!

All data from the Maintenance table:
Maintenance ID: 1, Computer ID: C003, Maintenance Date: 2025-03-19 10:00:00, Description: Hardware issue, screen replacement


In [ ]:
import mysql.connector

try:

    conn = mysql.connector.connect(
        host="localhost",
        user="root", 
        password=""
    )

    cursor = conn.cursor()
    cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

    cursor.execute("""
        INSERT INTO Maintenance (computer_id, maintenance_date, description)
        VALUES
        ('C003', '2025-03-19 10:00:00', 'Hardware issue, screen replacement')
    """)

    conn.commit()
    print("Data inserted into Maintenance table successfully!")

    # Display all data from Maintenance table
    cursor.execute("SELECT * FROM Maintenance")
    results = cursor.fetchall()

    print("\nAll data from the Maintenance table:")
    for row in results:
        print(f"Maintenance ID: {row[0]}, Computer ID: {row[1]}, Maintenance Date: {row[2]}, Description: {row[3]}")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Data inserted into Maintenance table successfully!

All data from the Maintenance table:
Maintenance ID: 1, Computer ID: C003, Maintenance Date: 2025-03-19 10:00:00, Description: Hardware issue, screen replacement
Maintenance ID: 2, Computer ID: C003, Maintenance Date: 2025-03-19 10:00:00, Description: Hardware issue, screen replacement


In [ ]:
import mysql.connector

try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root", 
        password=""
    )

    cursor = conn.cursor()

    cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

   
    cursor.execute("SELECT * FROM user_role WHERE role_id = 2")
    role_exists = cursor.fetchone()

    if not role_exists:
      
        cursor.execute("""
            INSERT INTO user_role (role_id, role_name) 
            VALUES (2, 'student')
        """)
        conn.commit()
        print("Role with role_id 2 (student) inserted successfully.")

    # Now update the role_id for the user with userId = 1
    cursor.execute("""
        UPDATE users
        SET role_id = 2  -- New role_id for 'student'
        WHERE userId = 1;  -- Condition to identify the user
    """)

    # Commit the changes to the database
    conn.commit()
    print("User role updated successfully!")

    # Retrieve and display all data from the users table
    cursor.execute("SELECT * FROM users")
    results = cursor.fetchall()

    print("\nAll data from the users table:")
    for row in results:
        print(f"User ID: {row[0]}, First Name: {row[1]}, Last Name: {row[2]}, Username: {row[3]}, Email: {row[4]}, Phone: {row[5]}, Role ID: {row[6]}, Status: {row[7]}")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Role with role_id 2 (student) inserted successfully.
User role updated successfully!

All data from the users table:


In [46]:
import mysql.connector

try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root", 
        password=""
    )

    cursor = conn.cursor()

    cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

    cursor.execute("SELECT * FROM users ORDER BY userId DESC")

    result = cursor.fetchall()

    for row in result:
        print(row)

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


(1, 'Emmanuel', 'Hakizimana', 'emmanuel', 'emmanuel@example.com', '1234567890', '$2b$12$Syi4Xe1Rn9KTUU92YsVHZ.N1teriBKMMVcSxxU0PtG1p.f0GEatwa', 2, 'Active')


In [47]:
import mysql.connector

try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root", 
        password=""
    )

    cursor = conn.cursor()

    cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

    cursor.execute("""
        SELECT users.userId, users.fname, users.lname, User_role.role_name
        FROM users
        INNER JOIN User_role
        ON users.role_id = User_role.role_id;
    """)

    result = cursor.fetchall()

    for row in result:
        print(row)

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


(1, 'Emmanuel', 'Hakizimana', 'student')


In [49]:
import mysql.connector

def deactivate_user(user_id):
    try:
        # Connect to the MySQL database
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password=""
        )

        cursor = conn.cursor()

        # Use the database
        cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

        # Update the user status to 'Inactive'
        cursor.execute("""
            UPDATE users
            SET status = 'Inactive'
            WHERE userId = %s
        """, (user_id,))

        conn.commit()

        print(f"User with userId {user_id} has been deactivated!")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Example usage
deactivate_user(1)


User with userId 1 has been deactivated!


In [ ]:
import mysql.connector

def update_computer_status(computer_id, new_status):
    try:
       
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password=""
        )

        cursor = conn.cursor()

        cursor.execute("USE HAKIZIMANA_Emmanuel_lab")

        # Update the status of the computer
        cursor.execute("""
            UPDATE Computers
            SET status = %s
            WHERE computer_id = %s
        """, (new_status, computer_id))

        conn.commit()

        print(f"Computer with computer_id {computer_id} status updated to {new_status}.\n")

        # Retrieve and display the updated data from the Computers table
        cursor.execute("SELECT * FROM Computers")
        results = cursor.fetchall()

        print("Updated data from the Computers table:")
        for row in results:
            print(f"Computer ID: {row[0]}, Status: {row[1]}")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Example usage
update_computer_status('C001', 'reserved')


Computer with computer_id C001 status updated to reserved.

Updated data from the Computers table:
Computer ID: C001, Status: reserved
Computer ID: C002, Status: reserved
Computer ID: C003, Status: under maintenance
